# REINFORCE

Also knwon as  `Monte Carlo Policy Gradient`, a **policy-gradient learning algorithm** with the following update rule:

$$
\theta_{t+1} = \theta_t + \alpha G_t \frac{\nabla_{\theta}\pi(A_t \given S_t, \theta_t)}{\pi(A_t \given S_t, \theta_t)}
$$

Note that each increment is proportional to the product of a return $G_t$ and a vector; the gradient of the probability
of taking the action taken divided by the probability of taking that action. The vector is the
direction in parameter space that most increases the probability of repeating the action $A_t$ on future
visits to state $S_t$. The update increases the parameter vector in this direction proportional to the
return, and inversely proportional to the action probability. The former makes sense because it causes
the parameter to move most in the directions that favor actions that yield the highest return. The latter
makes sense because otherwise actions that are selected frequently are at an advantage.

> Note that `REINFORCE` uses the complete return from time $t$, which includes all future rewards up until the end of the episode.